In [1]:
import os
import sys
import pandas as pd
import math
import re
import numpy as np

In [5]:
mmseqs2_tax_table = "/Users/michaeltisza/mike_tisza/sandbox/test_SRS_0803c/ct2_tmp/final_taxonomy/hallmark_proteins_align.tsv"

annotation_table = "/Users/michaeltisza/mike_tisza/sandbox/test_SRS_0803c/ct2_tmp/contig_gene_annotation_summary.pruned.tsv"

In [3]:
tax_df = pd.read_csv(mmseqs2_tax_table, header = None, sep = "\t",
                     names = ["query","target","pident","alnlen","evalue","theader","taxlineage"])\
                     .sort_values('evalue').drop_duplicates('query').query("evalue <= 1e-3")

In [6]:
gene_contig_df = pd.read_csv(annotation_table, sep = "\t")[['contig', 'chunk_name', 'gene_start',
                                                            'gene_stop', 'gene_name']]

In [7]:
chunk_wise_df = gene_contig_df.merge(tax_df, left_on = "gene_name", right_on = "query", how = "inner")

chunk_wise_df.head()

,contig,chunk_name,gene_start,gene_stop,gene_name,query,target,pident,alnlen,evalue,theader,taxlineage
0,test_SRS_0803c103,NaN,1,379,test_SRS_0803c103_9,test_SRS_0803c103_9,NP_813696.1,29.8,124,1.262000e-09,NP_813696.1 |major capsid protein [Lomovskayav...,d_Viruses;-_Duplodnaviria;k_Heunggongvirae;p_U...
1,test_SRS_0803c103,NaN,432,999,test_SRS_0803c103_10,test_SRS_0803c103_10,YP_010742650.1,49.9,145,1.335000e-35,YP_010742650.1 |putative prohead protease prot...,d_Viruses;-_Duplodnaviria;k_Heunggongvirae;p_U...
2,test_SRS_0803c103,NaN,1000,2233,test_SRS_0803c103_11,test_SRS_0803c103_11,NP_612832.1,28.6,331,5.704000e-34,NP_612832.1 |portal protein [Clostridium phage...,d_Viruses;-_Duplodnaviria;k_Heunggongvirae;p_U...
3,test_SRS_0803c103,NaN,2216,2573,test_SRS_0803c103_12,test_SRS_0803c103_12,YP_003090183.1,29.7,111,1.098000e-07,YP_003090183.1 |head closure protein [Burkhold...,d_Viruses;-_Duplodnaviria;k_Heunggongvirae;p_U...
4,test_SRS_0803c103,NaN,2559,2874,test_SRS_0803c103_13,test_SRS_0803c103_13,YP_008318486.1,29.4,101,4.014000e-06,YP_008318486.1 |head-tail connector protein [S...,d_Viruses;-_Duplodnaviria;k_Heunggongvirae;p_U...


In [21]:
#chunk_wise_df['genus_tag'] = np.where(chunk_wise_df['taxlineage'].str.contains(";g_"), 
#                                     chunk_wise_df['taxlineage'].str.find(";g_"), 
#                                     "not")
#chunk_wise_df['genus_end'] = np.where(chunk_wise_df['genus_tag'] != "not", 
#                                     chunk_wise_df['taxlineage'].str.find(";s_"), 
#                                     "not")


In [8]:
#.apply(lambda st: st[st.find("(")+1:st.find(")")])

chunk_wise_df['genus'] = np.where(chunk_wise_df['taxlineage'].str.contains(";g_"), 
                                     chunk_wise_df['taxlineage'].apply(lambda st: st[st.find(";g_")+3:].split(";")[0]), 
                                     "NA")

chunk_wise_df['family'] = np.where(chunk_wise_df['taxlineage'].str.contains(";f_"), 
                                     chunk_wise_df['taxlineage'].apply(lambda st: st[st.find(";f_")+3:].split(";")[0]), 
                                     "NA")

chunk_wise_df['order'] = np.where(chunk_wise_df['taxlineage'].str.contains(";o_"), 
                                     chunk_wise_df['taxlineage'].apply(lambda st: st[st.find(";o_")+3:].split(";")[0]), 
                                     "NA")

chunk_wise_df['taxclass'] = np.where(chunk_wise_df['taxlineage'].str.contains(";c_"), 
                                     chunk_wise_df['taxlineage'].apply(lambda st: st[st.find(";c_")+3:].split(";")[0]), 
                                     "NA")

chunk_wise_df['phylum'] = np.where(chunk_wise_df['taxlineage'].str.contains(";p_"), 
                                     chunk_wise_df['taxlineage'].apply(lambda st: st[st.find(";p_")+3:].split(";")[0]), 
                                     "NA")

In [9]:
group_chunk_df = chunk_wise_df.groupby(['contig', 'chunk_name'], dropna = False)

In [17]:
taxonomy_list = []

for name, group in group_chunk_df:
    #genus = is_unique(group['genus'])

    group_count = len(group.index)

    g_mode = group['genus'].agg(pd.Series.mode)

    g_mode_occur = len(group.query("genus == @g_mode[0]").index)

    g_mean_AAI = group.query("genus == @g_mode[0]")['pident'].mean()

    #
    f_mode = group['family'].agg(pd.Series.mode)

    f_mode_occur = len(group.query("family == @f_mode[0]").index)

    f_mean_AAI = group.query("family == @f_mode[0]")['pident'].mean()    
    #
    o_mode = group['order'].agg(pd.Series.mode)

    o_mode_occur = len(group.query("order == @o_mode[0]").index)

    o_mean_AAI = group.query("order == @o_mode[0]")['pident'].mean()  
    #
    c_mode = group['taxclass'].agg(pd.Series.mode)

    c_mode_occur = len(group.query("taxclass == @c_mode[0]").index)

    c_mean_AAI = group.query("taxclass == @c_mode[0]")['pident'].mean()  
    #
    p_mode = group['phylum'].agg(pd.Series.mode)

    p_mode_occur = len(group.query("phylum == @p_mode[0]").index)

    p_mean_AAI = group.query("phylum == @p_mode[0]")['pident'].mean()  

    if g_mode[0] != "NA" and (g_mode_occur / group_count) >= 0.75 and g_mean_AAI >= 80:
        taxonomy = g_mode[0]
        level = "genus"
        mean_AAI = g_mean_AAI
    elif f_mode[0] != "NA" and (f_mode_occur  / group_count) >= 0.75 and f_mean_AAI >= 35:
        taxonomy = f_mode[0]
        level = "family"
        mean_AAI = f_mean_AAI
    elif o_mode[0] != "NA" and (o_mode_occur  / group_count) >= 0.75 and o_mean_AAI >= 20:
        taxonomy = o_mode[0]
        level = "order"
        mean_AAI = o_mean_AAI
    elif c_mode[0] != "NA" and (c_mode_occur  / group_count) >= 0.75:
        taxonomy = c_mode[0]
        level = "class"
        mean_AAI = c_mean_AAI
    elif p_mode[0] != "NA" and (p_mode_occur  / group_count) >= 0.75:
        taxonomy = p_mode[0]
        level = "phylum"
        mean_AAI = p_mean_AAI
    else:
        taxonomy = "i give up"


    lineage_to_tax_S = np.where(group['taxlineage'].str.contains(taxonomy), 
                                     group['taxlineage'].apply(lambda st: st[:st.find(taxonomy)+len(taxonomy)]), 
                                     taxonomy)
    
    lineage_to_tax = lineage_to_tax_S[0]
    
    taxonomy_list.append([name[0], name[1], taxonomy, lineage_to_tax, level, mean_AAI])


taxonomy_call_df = pd.DataFrame(taxonomy_list, columns = ['contig', 'chunk_name', 'taxon', 'taxonomy_hierarchy', 'taxon_level', 'avg_hallmark_AAI_to_ref'])

taxonomy_call_df.head()

,contig,chunk_name,taxon,taxonomy_hierarchy,taxon_level,avg_hallmark_AAI_to_ref
0,test_SRS_0803c103,NaN,Caudoviricetes,d_Viruses;-_Duplodnaviria;k_Heunggongvirae;p_U...,class,33.342857
1,test_SRS_0803c1136,NaN,Caudoviricetes,d_Viruses;-_Duplodnaviria;k_Heunggongvirae;p_U...,class,53.600000
2,test_SRS_0803c1184,NaN,Caudoviricetes,d_Viruses;-_Duplodnaviria;k_Heunggongvirae;p_U...,class,38.200000
3,test_SRS_0803c1537,NaN,Peduoviridae,d_Viruses;-_Duplodnaviria;k_Heunggongvirae;p_U...,family,55.950000
4,test_SRS_0803c1610,NaN,Peduoviridae,d_Viruses;-_Duplodnaviria;k_Heunggongvirae;p_U...,family,49.450000


In [19]:

def taxon_decider(name, group, taxonomy_list):
    group_count = len(group.index)

    g_mode = group['genus'].agg(pd.Series.mode)

    g_mode_occur = len(group.query("genus == @g_mode[0]").index)

    g_mean_AAI = group.query("genus == @g_mode[0]")['pident'].mean()

    #
    f_mode = group['family'].agg(pd.Series.mode)

    f_mode_occur = len(group.query("family == @f_mode[0]").index)

    f_mean_AAI = group.query("family == @f_mode[0]")['pident'].mean()    
    #
    o_mode = group['order'].agg(pd.Series.mode)

    o_mode_occur = len(group.query("order == @o_mode[0]").index)

    o_mean_AAI = group.query("order == @o_mode[0]")['pident'].mean()  
    #
    c_mode = group['taxclass'].agg(pd.Series.mode)

    c_mode_occur = len(group.query("taxclass == @c_mode[0]").index)

    c_mean_AAI = group.query("taxclass == @c_mode[0]")['pident'].mean()  
    #
    p_mode = group['phylum'].agg(pd.Series.mode)

    p_mode_occur = len(group.query("phylum == @p_mode[0]").index)

    p_mean_AAI = group.query("phylum == @p_mode[0]")['pident'].mean()  

    if g_mode[0] != "NA" and (g_mode_occur / group_count) >= 0.75 and g_mean_AAI >= 80:
        taxonomy = g_mode[0]
        level = "genus"
        mean_AAI = g_mean_AAI
    elif f_mode[0] != "NA" and (f_mode_occur  / group_count) >= 0.75 and f_mean_AAI >= 35:
        taxonomy = f_mode[0]
        level = "family"
        mean_AAI = f_mean_AAI
    elif o_mode[0] != "NA" and (o_mode_occur  / group_count) >= 0.75 and o_mean_AAI >= 20:
        taxonomy = o_mode[0]
        level = "order"
        mean_AAI = o_mean_AAI
    elif c_mode[0] != "NA" and (c_mode_occur  / group_count) >= 0.75:
        taxonomy = c_mode[0]
        level = "class"
        mean_AAI = c_mean_AAI
    elif p_mode[0] != "NA" and (p_mode_occur  / group_count) >= 0.75:
        taxonomy = p_mode[0]
        level = "phylum"
        mean_AAI = p_mean_AAI
    else:
        taxonomy = "Unclassified Virus"


    lineage_to_tax_S = np.where(group['taxlineage'].str.contains(taxonomy), 
                                     group['taxlineage'].apply(lambda st: st[:st.find(taxonomy)+len(taxonomy)]), 
                                     taxonomy)
    
    lineage_to_tax = lineage_to_tax_S[0]
    
    return taxonomy_list.append([name[0], name[1], taxonomy, lineage_to_tax, level, mean_AAI])

taxonomy_list = []

for name, group in group_chunk_df:
    taxon_decider(name, group, taxonomy_list)
    
taxonomy_call_df = pd.DataFrame(taxonomy_list, columns = ['contig', 'chunk_name', 'taxon', 'taxonomy_hierarchy', 'taxon_level', 'avg_hallmark_AAI_to_ref'])

taxonomy_call_df.head()

,contig,chunk_name,taxon,taxonomy_hierarchy,taxon_level,avg_hallmark_AAI_to_ref
0,test_SRS_0803c103,NaN,Caudoviricetes,d_Viruses;-_Duplodnaviria;k_Heunggongvirae;p_U...,class,33.342857
1,test_SRS_0803c1136,NaN,Caudoviricetes,d_Viruses;-_Duplodnaviria;k_Heunggongvirae;p_U...,class,53.600000
2,test_SRS_0803c1184,NaN,Caudoviricetes,d_Viruses;-_Duplodnaviria;k_Heunggongvirae;p_U...,class,38.200000
3,test_SRS_0803c1537,NaN,Peduoviridae,d_Viruses;-_Duplodnaviria;k_Heunggongvirae;p_U...,family,55.950000
4,test_SRS_0803c1610,NaN,Peduoviridae,d_Viruses;-_Duplodnaviria;k_Heunggongvirae;p_U...,family,49.450000
